In [204]:
# Core Libraries
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns


sns.set_style("whitegrid")
np.random.seed(42) 

Conda env name: dubai-datathon-team4
python version 3.9 

In [205]:
# Load data
data = pd.read_csv("/Users/mac/Desktop/Dubai-Datathon-2025/team4/data/blood_transfusion.csv") 

# Preview
print("Shape:", data.shape)
data.head()

Shape: (15045, 57)


,patient_id,hadm_id,icustay_id,age,gender,race,weight,insurance,language,admission_type,...,on_diuretics,diuretic_type,ldh,bilirubin_total,bilirubin_direct,possible_hemolysis,in_hospital_mortality,dod,los_hospital_days,los_icu_days
0,10002114,27793700,34672098,56,M,UNKNOWN,64.0,Medicaid,English,OBSERVATION ADMIT,...,1,bumetanide,343.0,1.1,NaN,1,0,2162-12-11,14.708333,2.916667
1,10002155,28994087,31090461,80,F,WHITE,47.9,Medicare,English,EW EMER.,...,1,bumetanide,NaN,NaN,NaN,0,0,2131-03-10,5.875000,3.916667
2,10002428,23473524,35479615,80,F,WHITE,49.3,Medicare,English,EW EMER.,...,1,furosemide,NaN,NaN,NaN,0,0,NaN,11.000000,11.000000
3,10003400,20214994,32128372,72,F,BLACK/AFRICAN AMERICAN,94.1,Medicare,English,URGENT,...,1,furosemide,536.0,6.5,4.3,1,0,2137-09-02,23.208333,12.916667
4,10004401,29988601,32773003,82,M,WHITE,75.8,Medicare,English,EW EMER.,...,1,furosemide,NaN,NaN,NaN,0,0,2144-06-18,14.166667,10.625000


In [206]:
print(f"Loaded {len(data):,} patient records")
print(f"Columns: {data.columns.tolist()}")
print(f"\nDataframe shape: {data.shape}")
print(f"\nData types:\n{data.dtypes}")
print(data.info())

Loaded 15,045 patient records
Columns: ['patient_id', 'hadm_id', 'icustay_id', 'age', 'gender', 'race', 'weight', 'insurance', 'language', 'admission_type', 'primary_icd_code', 'icd_version', 'primary_icd_long_title', 'ongoing_bleeding', 'icu_admission_time', 'first_transfusion_time', 'time_to_first_transfusion_hours', 'early_transfusion', 'number_of_transfusions', 'units_first_transfusion', 'total_units_transfused', 'transfusion_during_vasopressor', 'sofa_score', 'heart_disease', 'kidney_disease', 'history_of_bleeding', 'sepsis', 'baseline_hemoglobin', 'pre_transfusion_hemoglobin', 'post_transfusion_hemoglobin', 'hemoglobin_ordered', 'how_many_times_hemoglobin_ordered', 'baseline_wbc', 'baseline_platelets', 'baseline_hematocrit', 'baseline_creatinine', 'baseline_spo2', 'baseline_sao2', 'spo2_measurement_count', 'baseline_bp_systolic', 'baseline_bp_diastolic', 'on_vasopressors', 'vasopressor_type', 'vasopressor_start_time', 'vasopressor_stop_time', 'vasopressor_duration_hours', 'succes

In [207]:
na_counts = data.isna().sum()

# Percentage of missing values per column
# Series[float] — same index, values = percentage of NaNs (0–100)
na_percent = (data.isna().mean() * 100).round(2)

# Combine both into a single DataFrame for readability
# DataFrame with two columns: 'n_missing' and 'missing_%'
missing_summary = pd.DataFrame({
    "n_missing": na_counts,
    "missing_%": na_percent
}).sort_values(by="missing_%", ascending=False)

print (missing_summary)

# Display top rows — shows columns with the most missingness first
print(f"Total columns: {data.shape[1]}, total rows: {data.shape[0]}")

                                   n_missing  missing_%
bilirubin_direct                       13634      90.62
baseline_sao2                          10904      72.48
ldh                                     8430      56.03
dod                                     7959      52.90
successful_weaning                      6449      42.86
vasopressor_duration_hours              6449      42.86
vasopressor_start_time                  6449      42.86
vasopressor_stop_time                   6449      42.86
bilirubin_total                         6174      41.04
weight                                  2637      17.53
pre_transfusion_hemoglobin              2618      17.40
baseline_wbc                            1936      12.87
baseline_platelets                      1895      12.60
baseline_hemoglobin                     1647      10.95
diuretic_type                           1484       9.86
baseline_hematocrit                     1440       9.57
baseline_bp_diastolic                    462    

In [208]:
# Handling missing values

# -------------------------------
# 1) Drop specified columns
# -------------------------------
cols_to_drop = [
    "bilirubin_direct",  # DOES NOT MATTER 
    "bilirubin_total",   # DOES NOT MATTER 
    "weight",            # DOES NOT MATTER 
    "ldh",               # MATTERS BUT not that relevant
    "dod",
]

# Keep only columns that actually exist to avoid KeyError
existing_to_drop = [c for c in cols_to_drop if c in data.columns]

# drop
data_irr_dropped = data.drop(columns=existing_to_drop)

print(f"Dropped columns: {existing_to_drop}")
print(f"Current number of patient records: {len(data_irr_dropped):,}")

Dropped columns: ['bilirubin_direct', 'bilirubin_total', 'weight', 'ldh', 'dod']
Current number of patient records: 15,045


In [209]:

# -----------------------------------------------
# Drop rows where any of these key columns are missing
# -----------------------------------------------



cols_to_check = [
    "pre_transfusion_hemoglobin",   # MATTER -> 17.40
    "baseline_wbc",                 # MATTER -> 12.87
    "baseline_platelets",           # MATTER -> 12.60
    "baseline_hemoglobin",          # MATTER -> 10.95
    "diuretic_type",                # NOT THAT IMPORTANT - only for patients 65% & already it s only 9.86% 
    "baseline_hematocrit",          # MATTER -> 9.57
    "baseline_bp_diastolic",
    "baseline_bp_systolic",
    "baseline_spo2",
    "post_transfusion_hemoglobin",
    "insurance",
    "language",
    "baseline_creatinine",
    "icd_version",
    "primary_icd_long_title",
    "primary_icd_code",
    "los_icu_days"
]


# Keep only columns that actually exist in the dataframe
existing_cols = [c for c in cols_to_check if c in data_irr_dropped.columns]

# Display how many rows will be dropped
n_before = len(data_irr_dropped)
n_missing_rows = data_irr_dropped[existing_cols].isna().any(axis=1).sum()

print(f"Rows before: {n_before}")
print(f"Rows with missing values in {existing_cols}: {n_missing_rows}")

# Drop rows with any NaN in these columns
data_irr_dropped_2 = data_irr_dropped.dropna(subset=existing_cols).reset_index(drop=True)

print(f"Rows after: {len(data_irr_dropped_2)} (dropped {n_missing_rows})")

Rows before: 15045
Rows with missing values in ['pre_transfusion_hemoglobin', 'baseline_wbc', 'baseline_platelets', 'baseline_hemoglobin', 'diuretic_type', 'baseline_hematocrit', 'baseline_bp_diastolic', 'baseline_bp_systolic', 'baseline_spo2', 'post_transfusion_hemoglobin', 'insurance', 'language', 'baseline_creatinine', 'icd_version', 'primary_icd_long_title', 'primary_icd_code', 'los_icu_days']: 5352
Rows after: 9693 (dropped 5352)


In [210]:
# -------------------------------
# 2) Baseline SaO2:
#    a) Make a table showing SaO2 missingness across race
#    b) Impute baseline_sao2 with baseline_spo2 when missing
# -------------------------------
if "race" not in data_irr_dropped_2.columns:
    raise KeyError("Column 'race' not found. Needed for missingness-by-race table.")

if "baseline_sao2" not in data_irr_dropped_2.columns:
    raise KeyError("Column 'baseline_sao2' not found.")

if "baseline_spo2" not in data_irr_dropped_2.columns:
    raise KeyError("Column 'baseline_spo2' not found to use for imputation of baseline_sao2.")

# Build a concise missingness table for SaO2 by race
tmp = (
    data_irr_dropped_2.groupby("race")
        .agg(
            total_rows=("baseline_sao2", "size"),
            n_missing_sao2=("baseline_sao2", lambda s: s.isna().sum())
        )
        .assign(missing_pct_sao2=lambda df: (df["n_missing_sao2"] / df["total_rows"] * 100).round(2))
        .sort_values("missing_pct_sao2", ascending=False)
)
print("\n=== SaO2 Missingness by Race ===")
display(tmp)

# Impute baseline_sao2 with baseline_spo2 where SaO2 is missing
# (Simple substitution per your guideline)
mask_sao2_na = data_irr_dropped_2["baseline_sao2"].isna()
n_before = mask_sao2_na.sum()

data_irr_dropped_2.loc[mask_sao2_na, "baseline_sao2"] = data_irr_dropped_2.loc[mask_sao2_na, "baseline_spo2"]

n_after = data_irr_dropped_2["baseline_sao2"].isna().sum()
print(f"\nImputed baseline_sao2 using baseline_spo2 for {n_before - n_after} rows (remaining NaNs: {n_after}).")



=== SaO2 Missingness by Race ===


,total_rows,n_missing_sao2,missing_pct_sao2
race,,,
ASIAN - KOREAN,8,8,100.00
WHITE - BRAZILIAN,14,12,85.71
BLACK/CARIBBEAN ISLAND,63,52,82.54
AMERICAN INDIAN/ALASKA NATIVE,25,20,80.00
SOUTH AMERICAN,9,7,77.78
HISPANIC/LATINO - CENTRAL AMERICAN,9,7,77.78
HISPANIC/LATINO - MEXICAN,9,7,77.78
HISPANIC/LATINO - GUATEMALAN,19,14,73.68
BLACK/AFRICAN,34,25,73.53



Imputed baseline_sao2 using baseline_spo2 for 6223 rows (remaining NaNs: 0).


In [211]:
data_irr_dropped_2 = data_irr_dropped_2.drop(columns="baseline_spo2")


In [212]:
final_df = data_irr_dropped_2 

In [213]:
print(f"Current number of patient records: {len(final_df):,}")

Current number of patient records: 9,693


In [214]:
%%time

from ydata_profiling import ProfileReport

profile = ProfileReport(final_df, title = "Report") 
profile.to_notebook_iframe()

# Save the report to an HTML file
profile.to_file("/Users/mac/Desktop/Dubai-Datathon-2025/team4/profile-report.html")

# note: takes around 20 seconds to run

In [215]:

predictors = ["age","gender","race","early_transfusion","sofa_score","sepsis",
                "admission_type","heart_disease","kidney_disease","history_of_bleeding",
                "baseline_hemoglobin","baseline_wbc","on_vasopressors"]

final_df[predictors].dtypes

age                      int64
gender                  object
race                    object
early_transfusion        int64
sofa_score               int64
sepsis                   int64
admission_type          object
heart_disease            int64
kidney_disease           int64
history_of_bleeding      int64
baseline_hemoglobin    float64
baseline_wbc           float64
on_vasopressors          int64
dtype: object

In [216]:
# drop_first=True avoids multicollinearity (drops one dummy per feature).
final_df = pd.get_dummies(final_df, columns=["gender", "race", "admission_type"], drop_first=True)

In [223]:
final_df["number_of_transfusions"]

0       4
1       6
2       2
3       4
4       3
       ..
9688    2
9689    8
9690    7
9691    1
9692    1
Name: number_of_transfusions, Length: 9693, dtype: int64

In [227]:
# -----------------------------------------------
# 1) Imports
# -----------------------------------------------
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# -----------------------------------------------
# 2) Setup: target & features
# -----------------------------------------------
TARGET = "number_of_transfusions"
df = final_df.copy()

# Drop rows with any NaNs (either in features or target)
df = df.dropna().copy()

# Drop rows where target missing or non-numeric
df = df[pd.notnull(df[TARGET])].copy()

# Ensure target numeric and non-negative
y_raw = df[TARGET].astype(float)
y_raw = np.clip(y_raw, a_min=0, a_max=None)

# -----------------------------------------------
# 3) Create bins for classification
# -----------------------------------------------
def make_quantile_bins(y, q):
    labels = [f"Q{i}" for i in range(1, q + 1)]
    y_b = pd.qcut(y, q=q, labels=labels, duplicates="drop")
    return y_b.astype("category")

y_binned = None
for q in (4, 3, 2):
    try:
        y_try = make_quantile_bins(y_raw, q)
        if y_try.value_counts().min() >= 2:
            y_binned = y_try
            break
    except Exception:
        pass

# Fallback: binary 0 vs >0
if y_binned is None:
    y_binned = pd.Series(np.where(y_raw == 0, "0", ">0"), index=df.index).astype("category")

print("Class counts after binning:")
print(y_binned.value_counts())

# -----------------------------------------------
# 4) Define X (all variables except target)
# -----------------------------------------------
X = df.drop(columns=[TARGET])

# Identify categorical vs numeric automatically
cat_cols = X.select_dtypes(include=["object", "category", "bool"]).columns.tolist()
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()

# -----------------------------------------------
# 5) Split safely
# -----------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y_binned, test_size=0.2, random_state=42, stratify=y_binned
)

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

# -----------------------------------------------
# 6) Build preprocessing + model pipeline
# -----------------------------------------------
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols),
        ("num", "passthrough", num_cols),
    ],
    remainder="drop",
)

clf = Pipeline(
    steps=[
        ("prep", preprocessor),
        ("logreg", LogisticRegression(max_iter=1000, class_weight="balanced", multi_class="auto"))
    ]
)

# -----------------------------------------------
# 7) Train & evaluate
# -----------------------------------------------
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("\nClassification report:")
print(classification_report(y_test, y_pred))

print("Confusion matrix:")
print(confusion_matrix(y_test, y_pred))

# Optional: show bin ranges
if set(y_binned.unique()) - {"0", ">0"}:
    desc = pd.concat(
        [y_raw.groupby(y_binned).min().rename("min"),
         y_raw.groupby(y_binned).max().rename("max"),
         y_binned.value_counts().rename("count")],
        axis=1
    ).sort_index()
    print("\nBin ranges (on original number_of_transfusions):")
    print(desc)


Class counts after binning:
number_of_transfusions
Q1    3709
Q3    1972
Q2     860
Name: count, dtype: int64
Train shape: (5232, 87), Test shape: (1309, 87)


/Users/mac/anaconda3/envs/dubai-datathon-team4/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Users/mac/anaconda3/envs/dubai-datathon-team4/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



Classification report:
              precision    recall  f1-score   support

          Q1       0.98      0.91      0.94       742
          Q2       0.58      0.88      0.70       172
          Q3       0.98      0.88      0.93       395

    accuracy                           0.90      1309
   macro avg       0.85      0.89      0.86      1309
weighted avg       0.93      0.90      0.91      1309

Confusion matrix:
[[678  63   1]
 [ 13 152   7]
 [  2  45 348]]

Bin ranges (on original number_of_transfusions):
                        min   max  count
number_of_transfusions                  
Q1                      1.0   2.0   3709
Q2                      3.0   3.0    860
Q3                      4.0  76.0   1972


/var/folders/sw/5f53r1m13rq2jww8lqgbks700000gn/T/ipykernel_17197/2424041185.py:106: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  [y_raw.groupby(y_binned).min().rename("min"),
/var/folders/sw/5f53r1m13rq2jww8lqgbks700000gn/T/ipykernel_17197/2424041185.py:107: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  y_raw.groupby(y_binned).max().rename("max"),
